In [1]:
#importation des dépendances
import numpy as np
import cv2 as cv
import glob
import pickle

# Etape1: Définition des coordonnées du monde réèl avec un motif en damier

#définition des dimensions du damier
chessboardSize = (9,6)
#la taille du cadre
frameSize = (640,480)

#condition d'arrèt de l'algorithme
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)


#Définir les coordonnées mondiales des points 3D
objp = np.zeros((chessboardSize[0] * chessboardSize[1], 3), np.float32)
objp[:,:2] = np.mgrid[0:chessboardSize[0],0:chessboardSize[1]].T.reshape(-1,2)

size_of_chessboard_squares_mm = 20
objp = objp * size_of_chessboard_squares_mm

#Tableaux pour stocker les points d'objet et les points
#d'image de toutes les images.
objpoints = [] # point 3D dans le monde réèl
imgpoints = [] # point 2D dans l'image

#Etape2: Définir les points d'objets et les points images en utilisant 
#les images du damier capturer  sous différents point de vue

#chemin des images
images = glob.glob('images/*.png')

for image in images:

  #Lecture de l'image
  img = cv.imread(image)
  gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

  #Trouver les coins du damier
  ret, corners = cv.findChessboardCorners(gray, chessboardSize, None)

  #Si le nombre de coins souhaité est détecté,nous affinons les coordonnées des pixels et les
  #afficher sur les images du damier

  if ret == True:

    objpoints.append(objp)
    # affiner les coordonnées des pixels pour des points 2D donnés.
    corners2 = cv.cornerSubPix(gray, corners, (11,11), (-1,-1), criteria)
    imgpoints.append(corners)

    cv.drawChessboardCorners(img, chessboardSize, corners2, ret)
    cv.imshow('img', img)
    cv.waitKey(1000)

cv.destroyAllWindows()


#Etape3: Calibrage de la caméra

ret, cameraMatrix, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, frameSize, None, None)

#stockage des résultats
print("Matrice de la caméra : \n")
print(cameraMatrix)
print("Distorsion : \n")
print(dist)
print("Rotation : \n")
print(rvecs)
print("Translation : \n")
print(tvecs)


# Ouverture du fichier en mode écriture
#with open('Resultat_Calibrage.txt', 'w') as file:
    # Enregistrement de la matrice de la caméra
   # file.write("Matrice de la caméra:\n")
   # file.write(np.array_str(cameraMatrix) + "\n\n")

    # Enregistrement des coefficients de distorsion
    #file.write("Distorsion:\n")
    #file.write(np.array_str(dist) + "\n\n")

    # Enregistrement des vecteurs de rotation
    #file.write("Rotation:\n")
    #for i, rvec in enumerate(rvecs):
        #file.write(f"RVec[{i}]: {np.array_str(rvec)}\n")
   # file.write("\n")

    # Enregistrement des vecteurs de translation
    #file.write("Translation:\n")
    #for i, tvec in enumerate(tvecs):
        #file.write(f"TVec[{i}]: {np.array_str(tvec)}\n")
    #file.write("\n")

# Confirmation que les données ont été écrites
#print("Les resultats du calibrage sont enregistré dans'Resultat_Calibrage.txt'")


#correction de la distorsion
#charger une nouvelle image 
 
img = cv.imread('distorsion.png')
h,  w = img.shape[:2]
newCameraMatrix, roi = cv.getOptimalNewCameraMatrix(cameraMatrix, dist, (w,h), 1, (w,h))

#utilisation de la fonction de non-distorsion de open-cv
dst = cv.undistort(img, cameraMatrix, dist, None, newCameraMatrix)

# recadrage de l'image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
cv.imwrite('caliResult1.png', dst)

# correction de la distorsion avec remaping
mapx, mapy = cv.initUndistortRectifyMap(cameraMatrix, dist, None, newCameraMatrix, (w,h), 5)
dst = cv.remap(img, mapx, mapy, cv.INTER_LINEAR)

# recadrage de l'image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
cv.imwrite('caliResult2.png', dst)

#la distance foclal:Elle prend en entrée la matrice de la caméra
#la taille de l’image, et la taille du capteur

# Dimensions du capteur en millimètres
apertureWidth = 3.6  # en mm
apertureHeight = 2.7  # en mm

fovx, fovy, focalLength, principalPoint, aspectRatio = cv.calibrationMatrixValues(
    cameraMatrix, (h,w), apertureWidth, apertureHeight
)
print(f"Champ de vision horizontal (fovx) : {fovx} degrés")
print(f"Champ de vision vertical (fovy) : {fovy} degrés")
print(f"Distance focale : {focalLength} mm")
print(f"Point principal : {principalPoint}")
print(f"Rapport d'aspect : {aspectRatio}")



# erreur de calibrage
mean_error = 0

for i in range(len(objpoints)):
    imgpoints2, _ = cv.projectPoints(objpoints[i], rvecs[i], tvecs[i], cameraMatrix, dist)
    error = cv.norm(imgpoints[i], imgpoints2, cv.NORM_L2)/len(imgpoints2)
    mean_error += error

print( "Erreur total: {}".format(mean_error/len(objpoints)))







Matrice de la caméra : 

[[729.95169509   0.         361.90146557]
 [  0.         728.93781891 195.40520574]
 [  0.           0.           1.        ]]
Distorsion : 

[[ 3.66051514e-02 -4.34127411e-01  7.12047202e-04  2.84757386e-03
   8.66772464e-01]]
Rotation : 

(array([[-0.34596439],
       [-0.04840967],
       [ 1.53634891]]), array([[-0.36028092],
       [ 0.29269959],
       [ 1.55980164]]), array([[-0.33813051],
       [ 0.11401313],
       [ 1.40630806]]), array([[-0.32227193],
       [ 0.14573453],
       [ 1.29902611]]), array([[-0.34356649],
       [ 0.10975389],
       [ 1.29229368]]), array([[-0.33905465],
       [ 0.10491085],
       [ 1.2600379 ]]), array([[-0.34263374],
       [ 0.10295392],
       [ 1.24286061]]), array([[-0.33317867],
       [ 0.06433238],
       [ 1.20853931]]), array([[-0.33516867],
       [ 0.05472312],
       [ 1.19458589]]), array([[-0.34642481],
       [ 0.04522718],
       [ 1.16154195]]), array([[-0.34467513],
       [ 0.02923426],
       [ 